# **Ziel: Entwicklung eines ML-Modells zur Nachfrageprognose**

In [37]:
!pip install -U gdown

import pandas as pd
import requests
import io


def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Hilfsfunktion zum Laden einer CSV-Datei über eine direkte URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(io.StringIO(response.text))

    # Verzeichnis der Datei-IDs zur besseren Übersichtlichkeit
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


## **Überprüfung der Region - Guayas**

Der Datensatz wurde mit den Filialinformationen verknüpft, um sicherzustellen, dass alle Einträge aus der Provinz Guayas stammen.

In [38]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/Data Science Sinem/Time Series/df_train_filtered_top3_final.pkl'

df_train = pd.read_pickle(path)

print(f"✅ Datei geladen: {len(df_train):,} Zeilen und {len(df_train.columns)} Spalten")
df_train.head()


In [ ]:
print(df_stores.columns)


In [ ]:
# Verknüpfen der Verkaufsdaten mit den Filialinformationen
df_merged = df_train.merge(df_stores[['store_nbr', 'state']], on='store_nbr', how='left')

print(f"✅ Nach Merge: {len(df_merged):,} Zeilen, {len(df_merged.columns)} Spalten")
df_merged.head()


In [ ]:
print(df_merged['state'].value_counts())


# **Filterung auf die Top-3-Familien**

Um die Analyse gezielt auf die wichtigsten Produktgruppen zu konzentrieren, wird der Datensatz auf die drei häufigsten Familien „GROCERY I“, „BEVERAGES“ und „CLEANING“ eingeschränkt.

In [ ]:
df_items

In [ ]:
# Definieren der Top-3-Familien
top_families = ['GROCERY I', 'BEVERAGES', 'CLEANING']

item_ids = df_items[df_items['family'].isin(top_families)]['item_nbr'].unique()

# Trainingsdaten filtern
df_train_filtered = df_train[df_train['item_nbr'].isin(item_ids)]

# Kontrolle
print(f"✅ Gefilterte Daten: {len(df_train_filtered):,} Zeilen")
print(df_items['family'].value_counts())


In [ ]:
df_train_filtered

## Der Datensatz wird auf das erste Quartal 2014 **(1. Januar bis 31. März)** begrenzt, um einen überschaubaren Zeitraum für das Modelltraining zu verwenden.

In [ ]:
# Zeitraum definieren
start_date = '2014-01-01'
end_date = '2014-03-31'

df_train_filtered = df_train_filtered[
    (df_train_filtered['date'] >= start_date) &
    (df_train_filtered['date'] <= end_date)
]

In [ ]:
df_train_filtered

## Im Rahmen des Feature Engineerings werden zeitbasierte Merkmale (Jahr, Monat, Wochentag), Lag-Features und gleitende Durchschnitte erstellt, um Trends und saisonale Effekte für XGBoost sichtbar zu machen.

In [ ]:
# Lag-Features (Verzögerungen) erstellen — vergangene Verkaufswerte
df_train_filtered = df_train_filtered.sort_values(['store_nbr', 'item_nbr', 'date'])

df_train_filtered['lag_1'] = df_train_filtered['unit_sales'].shift(1)
df_train_filtered['lag_7'] = df_train_filtered['unit_sales'].shift(7)
df_train_filtered['lag_30'] = df_train_filtered['unit_sales'].shift(30)


df_train_filtered['rolling_std_7'] = df_train_filtered['unit_sales'].shift(1).rolling(window=7).std()

# Fehlende Werte nach Shifts auffüllen (optional)
df_train_filtered = df_train_filtered.fillna(0)

df_train_filtered = df_train_filtered.set_index('date')

# Kontrolle
print("✅ Neue Features hinzugefügt:")
print(df_train_filtered.columns)
df_train_filtered

In [ ]:
df_merged = df_train_filtered.merge(df_stores, on='store_nbr', how='left')

df_merged = df_merged.merge(df_items, on='item_nbr', how='left')

print("✅ Daten erfolgreich zusammengeführt!")
print(f"➡️ Zeilen: {len(df_merged):,}")
print(f"➡️ Spalten: {len(df_merged.columns)}")
print("\n🧠 Neue Spaltenbeispiele:")
print(df_merged.columns.tolist()[:15])  # Zeigt die ersten Spaltennamen
print("\n📊 Stichprobe:")
display(df_merged.head(3))


# **Aufteilung Training/Test**

In [ ]:
import pandas as pd

# Datum sicherstellen
df_train_filtered.index = pd.to_datetime(df_train_filtered.index)
df_train_filtered = df_train_filtered.sort_index()

# Zeiträume festlegen
train_start = '2014-01-01'
train_end   = '2014-02-28'
test_start  = '2014-03-01'
test_end    = '2014-03-31'

# Masken auf Basis des Index erstellen
train_mask = (df_train_filtered.index >= train_start) & (df_train_filtered.index <= train_end)
test_mask  = (df_train_filtered.index >= test_start)  & (df_train_filtered.index <= test_end)

# Daten filtern
df_train = df_train_filtered.loc[train_mask]
df_test  = df_train_filtered.loc[test_mask]

# Kontrolle
print("✅ Trainingsdaten:", len(df_train),
      "Zeilen von", df_train.index.min(), "bis", df_train.index.max())

print("✅ Testdaten:", len(df_test),
      "Zeilen von", df_test.index.min(), "bis", df_test.index.max())


In [ ]:
target_col = 'unit_sales'

feature_cols = [
    'store_nbr',
    'item_nbr',
    'onpromotion',
    'z_score',
    'year',
    'month',
    'day_of_week',
    'unit_sales_7d_avg'
]

# Trainingsdaten trennen
X_train = df_train[feature_cols]
y_train = df_train[target_col]

# 🔹 Testdaten trennen
X_test = df_test[feature_cols]
y_test = df_test[target_col]

# Kontrolle
print("Feature-Matrix (Train):", X_train.shape)
print("Target (Train):", y_train.shape)
print("Feature-Matrix (Test):", X_test.shape)
print("Target (Test):", y_test.shape)


# **XGBoost-Modellaufbau**
zur Vorhersage von numerischen Werten in dem Fall Verkaufszahlen

## XGBoost-Baseline

n_estimators=250 = Anzahl der Bäume:
Das Modell besteht aus 250 Entscheidungsbäumen. Mehr Bäume = stärkeres Modell (aber langsamer).

learning_rate=0.1 = Lernrate:
Wie stark jeder neue Baum die Gesamtvorhersage korrigiert. Kleinere Werte lernen langsamer, aber stabiler.


max_depth=7 = Maximale Tiefe der Bäume:
Wie viele „Fragen“ ein Baum stellen darf. Tiefer = komplexer, aber riskanter (Overfitting).

random_state=42 = Zufallsstartwert:
Damit die Ergebnisse reproduzierbar bleiben.

n_jobs=-1 = Anzahl der Prozessorkerne:
-1 bedeutet: benutze alle verfügbaren Kerne, um schneller zu rechnen.

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=250,
    learning_rate=0.1,
    max_depth=7,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Modell trainiert! Anzahl Vorhersagen:", len(y_pred))


In [ ]:
import matplotlib.pyplot as plt

daily_sales = df_train_filtered[['unit_sales', 'unit_sales_7d_avg', 'rolling_std_7']].resample('D').mean()

plt.figure(figsize=(20, 8))
plt.plot(daily_sales.index, daily_sales['unit_sales'], label='unit_sales', color='black',linewidth=1, alpha=0.7)
plt.plot(daily_sales.index, daily_sales['unit_sales_7d_avg'], label='unit_sales_7d_avg', linewidth=2,  color='blue')
plt.plot(daily_sales.index, daily_sales['rolling_std_7'], label='rolling_std_7', linewidth=2,  color='magenta')
plt.title('Original Sales, 7-day Rolling Mean, and 7-day Rolling Std Dev')
plt.xlabel('Datum')
plt.ylabel('Verkäufe')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = pd.Series(y_pred, index=y_test.index)

print("Länge y_test:", len(y_test))
print("Länge y_pred:", len(y_pred))


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Absolut Error : {mae:.4f} units: Das Modell sagt sehr nah an den echten Werten voraus.")
print(f"Mean Squared Error : {mse:.4f} units: Es gibt einzelne größere Fehler, aber insgesamt ist das Modell stabil.")


## Bewertung meiner XGBoost-Baseline

Der gezeigte Code definiert eine Funktion zur Bewertung der Modellleistung anhand typischer Prognosemetriken.
Mit der Funktion forecast_metrics(y_true, y_pred) werden wichtige Kennzahlen berechnet, die zeigen, wie gut das XGBoost-Modell die Verkaufszahlen vorhersagt.

Dazu gehören:

- MAE (Mean Absolute Error): durchschnittlicher absoluter Fehler – misst die Genauigkeit.

- Bias: zeigt, ob das Modell systematisch über- oder unterschätzt.

- MAD / rMAD: mittlere absolute Abweichung und deren relatives Verhältnis.

- MAPE (Mean Absolute Percentage Error): prozentuale Abweichung der Vorhersagen vom tatsächlichen Wert.

- RMSE (Root Mean Squared Error): gibt stärkere Gewichtung auf größere Fehler.

Diese Metriken sind entscheidend, um die Qualität eines Prognosemodells objektiv zu bewerten.
Sie ermöglichen es, Stärken und Schwächen des Modells zu erkennen und bilden die Grundlage für die spätere Optimierung des XGBoost-Modells.

Der Code liefert den ersten, quantitativen Beweis, wie zuverlässig deine Baseline-Vorhersage wirklich ist – ein essenzieller Schritt im gesamten Machine-Learning-Projekt.

In [ ]:
import numpy as np

def forecast_metrics(y_true, y_pred):
    """
    Compute a common set of forecast-error statistics.

    Parameters
    ----------
    y_true : 1-D array-like
        Actual (ground-truth) values.
    y_pred : 1-D array-like
        Forecasted values.

    Returns
    -------
    dict
        Keys: 'MAE', 'Bias', 'MAD', 'rMAD', 'MAPE', 'RMSE'
    """
    y_true = np.asarray(y_true, dtype=float).flatten()
    y_pred = np.asarray(y_pred, dtype=float).flatten()

    errors       = y_true - y_pred
    abs_errors   = np.abs(errors)
    pct_errors   = abs_errors / np.where(y_true == 0, np.nan, y_true)

    mae   = abs_errors.mean()
    bias  = errors.mean()
    rmse  = np.sqrt((errors ** 2).mean())

    mad   = np.abs(y_true - y_true.mean()).mean()

    rmad  = mae / mad if mad else np.nan

    mape  = np.nanmean(pct_errors) * 100

    return {
        "MAE" : mae,
        "Bias": bias,
        "MAD" : mad,
        "rMAD": rmad,
        "MAPE": mape,
        "RMSE": rmse
    }

In [ ]:
metrics = forecast_metrics(y_test, y_pred)
for k, v in metrics.items():
    print(f"{k}: {v:,.2f}")

## Optimierung von XGBoost-Baseline: Hyperparamtertuning



In diesem Abschnitt wird die Leistungsfähigkeit des XGBoost-Modells durch gezieltes Hyperparametertuning verbessert.
Anstatt feste Parameter zu verwenden, wird hier mit einer RandomizedSearchCV eine Vielzahl möglicher Parameterkombinationen getestet, um jene Konfiguration zu finden, die den niedrigsten Fehler (RMSE) erzielt.

**Vorgehen:**

Parameterraum definieren:
Es werden verschiedene Wertebereiche für wichtige Parameter festgelegt, z. B.

- learning_rate: Lernrate des Modells

- max_depth: maximale Baumtiefe

- subsample & colsample_bytree: Anteil der Daten und Features pro Baum

- n_estimators: Anzahl der Entscheidungsbäume

Zeitreihen-Cross-Validation (TimeSeriesSplit):
Da es sich um Zeitreihendaten handelt, wird eine spezielle Validierung verwendet, die den zeitlichen Verlauf respektiert – das verhindert Datenleckagen.

RandomizedSearchCV:
Diese Methode wählt zufällig Kombinationen aus dem Parameterraster aus und testet sie, um die besten Parameter effizient zu finden (schneller als GridSearchCV).

Trainieren & Bewerten:
Mit den optimalen Parametern wird das finale Modell neu trainiert und anhand der zuvor definierten Metriken (MAE, RMSE, MAPE usw.) bewertet.

Visualisierung:
Die grafische Darstellung zeigt die tatsächlichen Verkaufszahlen im Vergleich zu den vorhergesagten Werten — ein wichtiger Schritt zur visuellen Bewertung der Modellqualität.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

xgb = XGBRegressor(objective='reg:squarederror')

In [ ]:
# Definition von Parameterraster
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'n_estimators': [100, 200, 300, 500]
}

tscv = TimeSeriesSplit()

random_search = RandomizedSearchCV(estimator=xgb,
																	 param_distributions=param_grid,
                                   cv=tscv,
                                   n_iter=10,
                                   scoring='neg_mean_squared_error')

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Best Parameters for XGBoost:", best_params)

final_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    learning_rate=best_params["learning_rate"],
    subsample=best_params["subsample"],
    colsample_bytree=best_params["colsample_bytree"],
    random_state=42
)

# Train the model
final_model.fit(X_train, y_train)

# Make predictions
y_pred = final_model.predict(X_test)

metrics = forecast_metrics(y_test, y_pred)
for k, v in metrics.items():
    print(f"{k}: {v:,.2f}")

# Visualize actual vs. predicted
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('XGBoost Forecast vs. Actual')
plt.show()


## MLflow-Setup für XGBoost

In diesem Abschnitt wird MLflow genutzt, um das Training und die Bewertung des XGBoost-Modells automatisch zu protokollieren.
Alle wichtigen Informationen – wie Parameter, Metriken, Modelle und Diagramme – werden gespeichert, damit die Experimente vergleichbar und reproduzierbar bleiben.

Der Code:

- erstellt ein MLflow-Experiment (XGBoost_Basislinie),

- trainiert das Modell und berechnet Prognosemetriken,

- speichert alle Ergebnisse und das Modell direkt in MLflow.

So wird sichergestellt, dass jede Modellversion dokumentiert ist und später leicht verglichen oder erneut geladen werden kann.
Ein zentraler Schritt für Nachvollziehbarkeit und sauberes Experiment-Tracking.

In [ ]:
!pip install mlflow


In [ ]:
# MLflow-Setup für XGBoost-Basislinie

import mlflow
import mlflow.xgboost
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor

# MLflow Setup
mlflow.set_experiment("XGBoost_Basislinie")  # Name des Experiments

# Rahmenparameter definiert
baseline_params = {
    "n_estimators": 250,
    "learning_rate": 0.1,
    "max_depth": 7,
    "random_state": 42,
    "n_jobs": -1,
    "objective": "reg:squarederror"
}

# Start meines 1. Laufs
with mlflow.start_run(run_name="Basis-XGB"):

    # Modellaufbau & Training
    model = XGBRegressor(**baseline_params)
    model.fit(X_train, y_train)

    # Vorhersagen
    y_pred = model.predict(X_test)

    # Metriken berechnen
    metrics = forecast_metrics(y_test, y_pred)

    # Parameter & Metriken in MLflow speichern
    mlflow.log_params(baseline_params)
    mlflow.log_metrics({k: float(v) for k, v in metrics.items()})

    # Prognose-Diagramm erstellen
    plt.figure(figsize=(8, 5))
    plt.plot(y_test, label="Wahre Werte", color="blue")
    plt.plot(y_pred, label="Vorhersage", color="orange")
    plt.title("XGBoost-Basislinie: Wahre Werte vs. Vorhersage")
    plt.xlabel("Zeitindex")
    plt.ylabel("Verkaufswert")
    plt.legend()
    plt.tight_layout()

    # Diagramm speichern und als Artefakt hochladen
    plt.savefig("forecast_plot.png")
    mlflow.log_artifact("forecast_plot.png")
    plt.close()

    # Modell selbst speichern
    mlflow.xgboost.log_model(model, artifact_path="xgb_model_baseline")

    # Tags setzen
    mlflow.set_tag("stage", "baseline")
    mlflow.set_tag("model_type", "XGBoost")

    print("✅ MLflow-Basislauf abgeschlossen!")
    print("🔍 Ergebnisse im MLflow-UI anzeigen mit:  mlflow ui")

In [ ]:
import mlflow
import pandas as pd

# Dein Experiment-Name aus dem Code
exp_name = "XGBoost_Basislinie"

# Experiment-ID holen
exp = mlflow.get_experiment_by_name(exp_name)

# Alle Runs als DataFrame ziehen
runs_df = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    output_format="pandas"
)

# Nur die wichtigsten Spalten zeigen
cols = [
    "run_id", "tags.stage", "params.n_estimators", "params.learning_rate",
    "params.max_depth", "metrics.MAE", "metrics.RMSE", "metrics.MAPE"
]
display(runs_df[cols].sort_values("metrics.RMSE"))


2. Lauf – MLflow für optimiertes XGBoost-Modell

In [ ]:
import mlflow
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

# Zweiter MLflow-Lauf: Optimiertes Modell
with mlflow.start_run(run_name="Optimiertes-XGB"):

    # Trainiere Modell mit den besten Parametern aus RandomizedSearchCV
    optimized_model = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=best_params["n_estimators"],
        max_depth=best_params["max_depth"],
        learning_rate=best_params["learning_rate"],
        subsample=best_params["subsample"],
        colsample_bytree=best_params["colsample_bytree"],
        random_state=42
    )

    optimized_model.fit(X_train, y_train)
    y_pred_opt = optimized_model.predict(X_test)

    # Berechne Metriken
    metrics_opt = forecast_metrics(y_test, y_pred_opt)

    # Logge Parameter und Metriken in MLflow
    mlflow.log_params(best_params)
    mlflow.log_metrics(metrics_opt)

    # Erstelle Vergleichsdiagramm (tatsächliche vs. vorhergesagte Werte)
    plt.figure(figsize=(8, 5))
    plt.plot(y_test.values[:200], label="Wahre Werte", color="blue")
    plt.plot(y_pred_opt[:200], label="Optimierte Vorhersage", color="green")
    plt.title("Optimiertes XGBoost-Modell: Wahr vs. Vorhergesagt")
    plt.xlabel("Zeitindex")
    plt.ylabel("Verkaufswert")
    plt.legend()
    plt.tight_layout()
    plt.savefig("optimized_forecast_plot.png")

    # Logge Diagramm und Modell
    mlflow.log_artifact("optimized_forecast_plot.png")
    mlflow.xgboost.log_model(optimized_model, artifact_path="xgb_model_optimized")

    # Setze Tags (optional, für Übersicht)
    mlflow.set_tag("stage", "optimized")
    mlflow.set_tag("model_type", "XGBoost")

    print("✅ Zweiter MLflow-Lauf abgeschlossen! (Optimiertes Modell)")


## 📊Vergleich Baseline vs. Optimiertes Modell: XGBoost

In diesem Schritt werden die Ergebnisse des Basismodells und des optimierten XGBoost-Modells direkt miteinander verglichen.
Dazu werden die gespeicherten MLflow-Runs geladen und die wichtigsten Metriken (z. B. RMSE, MAE, MAPE) ausgewertet.

Die Balkengrafik zeigt die Unterschiede in der Modellleistung auf einen Blick – insbesondere beim RMSE-Wert, der die Prognosegenauigkeit misst.
So lässt sich leicht erkennen, ob das Tuning tatsächlich zu einer Verbesserung der Vorhersagequalität geführt hat.

Ein wichtiger Schritt, um den Erfolg des Modelltrainings objektiv zu bewerten.

In [ ]:
import pandas as pd

# Experiment-Name aus deinen Runs
exp_name = "XGBoost_Basislinie"

# Experiment-ID holen
exp = mlflow.get_experiment_by_name(exp_name)

# Alle Runs laden
runs_df = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    output_format="pandas"
)

# Nur wichtige Spalten anzeigen
cols = [
    "tags.stage",
    "params.n_estimators",
    "params.learning_rate",
    "params.max_depth",
    "metrics.MAE",
    "metrics.RMSE",
    "metrics.MAPE"
]

comparison_df = runs_df[cols].sort_values("metrics.RMSE").reset_index(drop=True)
display(comparison_df)


### Visuelle Gegenüberstellung (Barplot der Metriken)

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(comparison_df["tags.stage"], comparison_df["metrics.RMSE"], color=["orange", "green"])
plt.title("Vergleich: RMSE Baseline vs. Optimiert")
plt.ylabel("RMSE")
plt.xlabel("Modell")
plt.show()


In diesem Schritt wird aus allen gespeicherten MLflow-Runs automatisch das Modell mit dem besten RMSE-Wert ermittelt – also jenes, das die genauesten Vorhersagen liefert.

Der Code durchsucht die gespeicherten Ergebnisse, gibt die beste Run-ID aus und lädt das entsprechende Modell direkt aus MLflow.
Dadurch kannst du das leistungsstärkste Modell jederzeit wiederverwenden, ohne es neu trainieren zu müssen.

Ein wichtiger Schritt für Effizienz und Nachvollziehbarkeit im Modellmanagement.

In [ ]:
import mlflow
import mlflow.xgboost
import pandas as pd

# Name deines MLflow-Experiments
exp_name = "XGBoost_Basislinie"

# Experiment holen
exp = mlflow.get_experiment_by_name(exp_name)

# Alle Runs abrufen
runs_df = mlflow.search_runs(
    experiment_ids=[exp.experiment_id],
    output_format="pandas"
)

# Besten Run anhand des niedrigsten RMSE finden
best_run = runs_df.loc[runs_df["metrics.RMSE"].idxmin()]
best_run_id = best_run["run_id"]

print("🏆 Bestes Modell gefunden!")
print(f"Stage: {best_run['tags.stage']}")
print(f"RMSE: {best_run['metrics.RMSE']:.4f}")
print(f"Run-ID: {best_run_id}")


In [ ]:
# Modell aus MLflow laden
model_uri = f"runs:/{best_run_id}/xgb_model_optimized"  # oder xgb_model_baseline, falls du das willst
best_model = mlflow.xgboost.load_model(model_uri)

print("✅ Modell erfolgreich geladen!")
